<a href="https://colab.research.google.com/github/Rabia-creator/RabiaSaqib_DL_assignments_Q2_batch35/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
import tensorboard
from pylab import rcParams

In [ ]:
from google.colab import files
uploaded = files.upload()
df=pd.read_csv("creditcard.csv")

Saving creditcard.csv to creditcard.csv


In [1]:
df.head(10)

NameError: ignored

In [4]:
df.info()

NameError: ignored

In [7]:
df.describe()

NameError: ignored

In [ ]:
#configuring module
%matplotlib inline
sns.set(style='whitegrid',palette='muted',font_scale=1.5)
rcParams['figure.figsize'] = 14,8
Random_Seed=42

In [ ]:
# renaming columns
df.rename(columns={'class': "Fraud"},inplace=True)

In [ ]:
df.shape

In [ ]:
#checking null values
df.isnull().sum()

In [ ]:
#checking correlations
corr= df.corr()

#generates a mask for upper triangle
 mask= np.zeros_like(corr, dtype=np.bool)
 mask[np.triu_indices_from(mask)] = True

#figure size
f,ax= plt.subplots(figsize=(20, 10))

#generate a custom diverging colormap
cmap= sns.diverging_palette(220,10, as_cmap=True)

#draw the heatmap with the mask  and correct aspect ratio
sns.heatmap(corr,mask=mask, cmap='CMRmap_r',vmax=.3, center=0,square=True,linewidths=.5,cbar_kws={'shrink':.5})

In [ ]:
# Transaction class Distribution plot
labels=['valid','Fraud']
plt.figure(figsize =(10,10))
count_classes=pd.value_counts(df['Fraud'],sort =True)
count_classes.plot(kind='bar', rot=0, color ='red', alpha=0.5)
plt.xticks(range(2), labels)
plt.title("Transaction Class Distribution Graph")
plt.xlabel("Class")
plt.ylabel("Number of Observations")
plt.show()

In [ ]:
len(df)


In [ ]:
len_fraud = len(df[df.Fraud == 1])
len_fraud

In [ ]:
len_valid = 284807-492
len_valid

In [ ]:
perc_fraud=(len_fraud*100)/ len(df)
perc_fraud

In [ ]:
# making datafarames of valid and fraud transactions to check important stats
Fraud_df = df[df.Fraud == 1]
Valid_df = df[df.Fraud == 0]

In [ ]:
# checking important stats of Amouont used  in Fraud Transactions
Fraud_df.Amount.describe()

In [ ]:
# Amount per transaction by class
f,(ax1, ax2) = plt.subplots(2,1, sharex=True)
f.suptitle('Amoount Per Transaction Graph')
bins=50
ax1.hist(Fraud_df.Amoount, bins=bins, color='blue')
ax1.set_title('Fraud Transactions')

ax2.hist(Valid_df.Amouont,bins=bins, color='red',alpha = 0.5)
ax2.set_title('Valid Transactions')

plt.xlabel('Amount($)')
plt.ylabel('Number of Transaction')
plt.xlim((0,20000))
plt.yscale('log')
plt.show()

In [ ]:
#3 time of Transaction Vs amount by class
f,(ax1,ax2)= plt.subplots(2,1,sharex = True)
f.suptitle('time of Transactions Vs Amount of Transaction Graph')

ax1.scatter(Fraud_df.Time, Fraud_df.Amount)
ax1.set_title('Fraud Transactions')

ax2.scatter(Valid_df.Time,Valid_df.Amount)
ax2.set_title('Valid Transaction')

plt.xlabel('Time (in seconds)')
plt.ylabel('Amount')
plt.show()

In [ ]:
# normalize data
#we need to normalize 2 features (amount and time) 
from sklearn.preprocessing import StandardScaler
df_norm=df
df_norm['Time'] = StandardScaler().fit_transform(df_norm['time'].values.reshape(-1,1))
df_norm['Amount'] = StandardScaler().fit_transform(df_norm['Amount'].values.reshape(-1,1))

In [ ]:
# split dataset into training and testing
#Train Data

train_data=df_norm.iloc[:199365,:]
train_data.shape

In [ ]:
#test Data
test_data=df_norm.iloc[199365:,:]
test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
# shuffle train data
train_data=train_data.sample(frac=1)
train_data.head()

In [ ]:
# shuffle test data
test_data=test_data.sample(frac=1)
test_data.head()

In [ ]:
#dropping labels from training and testing data
train_labels=train_data.pop('Fraud')
# train_labels= train_labels.values

test_labels= test_data.pop('Fraud')
#test_labels=test_labels.values

In [ ]:
train_labels

In [ ]:
test_labels

In [ ]:
# building Base Model
model=Sequential()
model.add(Dense(10,activation='relu',input_shape = [len(train_data.keys())))
model.add(Dense(8, activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimzer='adam', loss= 'binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, Tensorboard
checkpointer=ModelCheckpoint(filepath= "CreditCardFraudDetectionModel.h5" , verbose=0, save_best_only = True)
tensorboard=Tensorboard(log_dir ='./logs',histogram_freq = 0, write_graph = True, write_images = True)

In [ ]:
#fitting base model on 100 epochs
history=model.fit(train_data,train_labels,epochs=100, batch_size=32, shuffle=True,validation_split=0.2, verbose= 1,call_backs=[checkpointer,tensorboard]).history

In [ ]:
# storing history in a data frame
history_df= pd.DAtaFrame(history)
history_df.tail()

In [ ]:
#plotting loss and accuracy
plt.plot(history_df['loss'])
plt.plot(history_df['Accuracy'])
plt.plot(history_df)['validation loss']
plt.plot(history_df['validtion Accuracy'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training_loss','Training_Accuracy,','valodation_loss','validation_Accuracy'],loc = 'upper right');

In [ ]:
# making predictions on TestData
predictions= model.predict(test_data)
predictions= predictions.flatten()

In [ ]:
predictions

In [ ]:
#testing model on testing data
testing= model.evaluate(test_data,test_labels)

In [ ]:
print(f"Test Loss: {testing[0]}")
print(f"Test Accuracy: {testing[1]}")

In [ ]:
# plotting Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

#y_pred =model.predict(X_test)

y_test = pd.DataFrame(testing)
cm = confusion_matrix(test_labels,predictions.round())
sns.heatmap(cm, annot=True,fmt='.Of' , cmap='cividis_r')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#building secong model for overcoming unbalanced data problem


In [ ]:
#standardizing the amount Feature
df['NormAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
df = df.drop(['Time','Amount'], axis=1)

In [ ]:
# SEparating test data
X = df.iloc[:,df.columns != 'class']
y=df.iloc[:,df.columns == 'Class']

In [ ]:
#importing SMOTE
from imblearn.over_sampling import SMOTE
np.random.seed(9)

In [ ]:
## counting the number of fraud and valid transactions
all_records = len(df)
number_records_fraud=len(df[df.Class ==1])
print(f"No of transactions: {all_records} , Fraud transactions: {number_records_fraud}")

In [ ]:
#Applying SMOTE
X_resample , y_resample =SMOTE().fit_sample(X,y.values.ravel())

In [ ]:
#transforming in to pandas DataFrame
y_resample = pd.DataFrame(y_resample)
X_resample = pd.DataFrame(X_resample)

In [ ]:
#splitting the dataset into training and testing
X_train,X_test,Y_train,Y_test = train_test_split(X_resample, y_resample, test_size=0.3, random_state=0)

In [ ]:
# making second model
model2=Sequential()
model2.add(Dense(16,kernal_initializer='uniform',activation='relu',input_shape=(29,)))
model2.add(Dense(18,kernal_initializer='unifrom', activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(20, kernal_initializer= 'uniform' ,activation='relu'))
model2.add(Dense(24,kernal_initializer='uniform' , activation='relu'))
model2.add(Dense(1, kernal_initializer='uniform',activation='sigmoid'))
model2.summary()

In [ ]:
#compilation step
model2.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
#fitting second model on 5 epochs
model2.fit(np.array(X_train),np.array(Y_train),batch_size=15,epoch=5)

In [ ]:
#testing second model on test data
score= model2.evaluate(np.array(X_test),np.array(Y_test), batch_size=128)
print('\nTesting Score is: ', score[1]*100, '%')

In [ ]:
#defination function for confusion matrix

import intertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,precession_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix'
                          cmap=plt.cm.Blues):
  ## this function prints and plots the confusion matrix. Normalization can be applied by setting  'normalize=True

  plt.imshow(cm, interpolation= 'nearest' , cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks=np.arange(len(classes))
  plot.xticks(tick_marks, classes , rotation=0)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm= cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # print('Normalized confusion matrix')
    else:
      #1 print('confusion matrix ,without normalization')

      # print(cm)

      thresh = cm.max() / 2.
      for  i , j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.textj, i cm[i,j], horizontalalignment = 'center', color='white' if cm[i,j] > thresh else 'black')

        plt.tight_layout()
        plt.ylabel('True Label')
        plt.xlabel('predicted Label')

In [ ]:
y_pred = model2.predict(np.array(X_test))

In [ ]:
Y_test = pd.DataFrame(Y_test)
Y_test.shape

In [ ]:
#plotting Confusion Matrix
cnf_matrix = confusion_matrix(Y_test,y_pred.round())
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: " , cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# plot non_normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix,classes=class_names,title='Confusion matrix')
plt.show()

In [ ]:
y_pred = model2.predict(np.array(X))

In [ ]:
# plotting  Confusion Matrix

cnf_matrix= confusion_matrix(y,y_pred.round())
np.set_printoptions(precision=2)

print("recall metric in the testing dataset: ", cnf_matrix[1,1]/cnf_matrix[1,0]+cnf_matrix[1,1]))

#plot non normalized confusion matrix
class_names= [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names ,title ='Confusion matrix')
plt.show()

In [ ]:
#save model
model2.save('CreditCardFraudDetectionModel.h5')